# Agenda of this Case Study
Derive insights from the data, in doing so we will
- learn to use python to fetch, manipulate, see and understand data
- get familiar with some common python libraries and functions we use for EDA

# Context of the dataset we are going to use
Each year, we at Stack Overflow ask the developer community about everything from their favorite technologies to their job preferences. This year marks the eighth year we’ve published our Annual Developer Survey results—with the largest number of respondents yet. Over 100,000 developers took the 30-minute survey in January 2018.

This year, we covered a few new topics ranging from artificial intelligence to ethics in coding. We also found that underrepresented groups in tech responded to our survey at even lower rates than we would expect from their participation in the workforce. Want to dive into the results yourself and see what you can learn about salaries or machine learning or diversity in tech? We look forward to seeing what you find!

# Content
This 2018 Developer Survey results are organized on Kaggle in two tables:

survey_results_public contains the main survey results, one respondent per row and one column per question

survey_results_schema contains each column name from the main results along with the question text corresponding to that column

There are 98,855 responses in this public data release. These responses are what we consider “qualified” for analytical purposes based on completion and time spent on the survey and included at least one non-PII question. Approximately 20,000 responses were started but not included here because respondents did not answer enough questions, or only answered questions with personally identifying information. Of the qualified responses, 67,441 completed the entire survey.

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

In [3]:
df = pd.read_csv('survey_results_public.csv')
questions = pd.read_csv('survey_results_schema.csv')

FileNotFoundError: [Errno 2] File survey_results_public.csv does not exist: 'survey_results_public.csv'

In [ ]:
print(df.shape)
df.head() 

- we got an error above, stating that there is a mix of data types in certain columns
- From the data above we can see that the column 'Company Size' has numbers and strings 
- Similarly it is the case for certain other columns
- The warning message says either specify the dtype of each and every column or set low_memory = False. The latter is more feasible for this dataset having 129 columns

In [ ]:
df = pd.read_csv('survey_results_public.csv', low_memory = False)  # We do not get the same warning

- Most warnings are often harmless but it is necessary to be cognizant of the same
- The way to deal with these warnings is often documented in the library that we use


_Now let's check out a few questions from the survey schema data_

In [ ]:
questions.head() 

- We can see how the table is structured
- Each column in df is essentially a question and the column name is an alias for that question
- We can see any question in its entirety by featching it from the questions dataframe

In [ ]:
questions.loc[4,'QuestionText']  

- We used .loc[ ] to fetch a specific value from the dataframe
- But what if we did not know the position of a certain column name whose question we want to fetch?

In [ ]:
questions.set_index('Column', inplace=True)
questions.head()

- Now that the index has actual names, we can use .loc[ ] to index based on the column values

In [ ]:
questions.loc['Student', 'QuestionText']

- Feel free to explore what other column names in 'df' mean by fetching their respective questions from the 'questions' dataframe


_One of the first few things we do to understand the data is look at its info()_

In [ ]:
df.info()

- Of the 129 columns, 41 are float type, 1 is integer type and the rest 87 are object type data
- We have 98855 rows in total
- And the amount of memory this dataset is occupying on the RAM is 97.3MB

_Here after, we will proceed with exploring and understanding the data without any obvious comments like we did earlier_

In [ ]:
df.isna().apply(pd.value_counts)

- Number of missing values in each of the columns
- Treating missing values is beyond the scope of this notebook, hence we will ignore it for now

In [ ]:
np.array(df.columns)

In [ ]:
df.Country.value_counts().head()

- For convinence, we have only displayed the top five countries
- To get a broader picture, we can create a barplot

In [ ]:
df.Country.value_counts().head(25).plot(kind = 'bar', edgecolor = 'black', color = 'lightblue', figsize = (15,5))
plt.xlabel('Country')
plt.ylabel('Number of users')
plt.title('Top 25 countries in terms of userbase')
plt.show()

- The above are top 25 countries in terms of userbase
- In the above plot, we can add another detail and make a stacked barchart

In [ ]:
top25 = df.loc[df.Country.isin(df.Country.value_counts().head(25).index), :]

# df.Country.value_counts().head(25).index gives the list of top 25 countries,
#isin() checks if any of the list elements is present in the df.Country series


pd.crosstab(top25.Country, top25.OpenSource).plot(kind = 'bar', edgecolor = 'black',
                                             color = ['lightblue', 'lightgreen'], figsize = (15,5))
plt.xlabel('Country')
plt.ylabel('Number of users')
plt.title('Top 25 countries in terms of userbase, segregated by their contribution to open source projects')
plt.show()

In [ ]:
pd.crosstab(top25.Country, top25.OpenSource)

- The green bars indicate the population that contributes to open source projects and the blue indicates the rest
- If we want to the bars to be sorted the same way as the earlier plot, we can index the crosstabdata in that order

In [ ]:
pd.crosstab(top25.Country, top25.OpenSource).loc[df.Country.value_counts().head(25).index,:].plot(kind = 'bar', 
                                                                                                  edgecolor = 'black',
                                                                                                  color = ['lightblue',
                                                                                                           'lightgreen'],
                                                                                                  figsize = (15,5))


plt.xlabel('Country')
plt.ylabel('Number of users')
plt.title('Top 25 countries in terms of userbase, segregated by their contribution to open source projects')
plt.show()

- Proportionally, Indian users seem to be contributing more for open source projects than most other countries
- In case of Turky, the two categories are almost the same
- In case of Iran, the open source contributors are more compared to the ones that do not contribute

**Task for student**: Write a piece of code to plot the number of users who code as a hobby vs number of users who do not, by country (consider the top 10 countries in terms of user base).  use: "Hobby" column.

_Let's explore the various job titles that users hold_

In [ ]:
df.DevType.unique()

- A lot of entries have composite job titles
- We have to write some additional steps to seperate the titles

In [ ]:
df.DevType.apply(lambda x: x.split(sep = ';'))

- The reason for the above error is the presence of NaN values in the data
- We can avoid it by using **try & except** block

In [ ]:
def split_title(title):
    
    try:
        return title.split(sep = ';')
    except:
        return []

In [ ]:

titles = df.DevType.apply(lambda x: split_title(x))
titles.head()

- What we are getting in the above code is a series of lists
- The next step would be to combine these lists

In [ ]:
title_list = []

for composite_title in df.DevType:
    
    title_list.extend(split_title(composite_title))

In [ ]:
len(np.unique(title_list))

- The above list contains all the job titles of the user base
- There are 20 unique job titles in the user pool
- Beyond just the job titles we can also get the count of individual job titles and see the proportion of each job title in the user pool

In [ ]:
series = pd.Series(title_list).value_counts()
 
explode = np.zeros(len(series))

indices = series.index.isin(['Data or business analyst', 'Data scientist or machine learning specialist'])

explode[indices] = 0.1

plt.figure(figsize=(15,15))

plt.pie(series, labels = series.index, autopct='%.1f%%',shadow = True, explode = explode, textprops={'fontsize': 14})

plt.show()

- The proportion of users who are in analytics or ML field are about 5.3%
- Close to 50% of the cohort are web developers
- You can observe the proportion other jobs 

**code explanation**: We got the frequency of job titles from title_list, by converting it to series and using the pd.value_counts method. Using these frequency counts, we made a pie chart. To pop the two specific job titles, we got a boolean array of series where the value is equal to the specific job title. We used this boolean array to index an array of zeros and replace the values in the array with a non-zero number

_Let's take a subset of the data that describes only the analysts_

In [ ]:
analysts = df.loc[df.DevType.isin(['Data or business analyst', 'Data scientist or machine learning specialist']), :]

In [ ]:
analysts.shape

In [ ]:
analysts.Age.value_counts().plot.barh().invert_yaxis()

- A significant majority of the data science professionals are of the age group 25-34

In [ ]:
import seaborn as sns

plt.figure(figsize=(9,5))
sns.heatmap(pd.crosstab(analysts.JobSatisfaction, analysts.Age, normalize = 'columns'), cmap = 'Blues', annot = True)
plt.show()

- 67% of the majority age-group i.e, 18-24, are quiet satisfied with their jobs
- Age groups 45-54 and 55-64 seem to be dissatisfied with their data science jobs

In [ ]:
salaries = analysts.loc[:,['Salary', 'SalaryType', 'Currency', 'ConvertedSalary']].dropna()
salaries.head()

In [ ]:
salaries.SalaryType.value_counts()

In [ ]:
salaries['monthly_salary'] = salaries.ConvertedSalary

for i in salaries.index:
    if salaries.loc[i, 'SalaryType'] == 'Yearly':
        salaries.loc[i, 'monthly_salary'] = salaries.loc[i, 'ConvertedSalary']/12
    elif salaries.loc[i, 'SalaryType'] == 'Weekly':
        salaries.loc[i, 'monthly_salary'] = salaries.loc[i, 'ConvertedSalary']*4

In [ ]:
salaries.head()

In [ ]:
fig, ax = plt.subplots(1,2, sharex = True)

fig.set_figheight(5)
fig.set_figwidth(20)
sns.boxplot(salaries.monthly_salary, ax= ax[0])
sns.distplot(salaries.monthly_salary, ax = ax[1])
plt.show()

- Histogram in this case does not give much information because of the extreme skew in the distribution
- The data is highly skewed. Hence, we will look at the numbers

In [ ]:
salaries.monthly_salary.describe().apply(lambda x: format(x, '.2f'))

- 75% of the people have salaries less than 19140 USD and the median salary for a data scientist is 7750 USD

In [ ]:
salaries['Salary'] = salaries.Salary.apply(lambda x: x.replace(',', '')).apply(float)

In [ ]:
Indian_salaries = salaries.loc[salaries.Currency.apply(lambda x: 'Indian' in x), :]
Indian_salaries['monthly_salary'] = Indian_salaries.Salary

for i in Indian_salaries.index:
    if Indian_salaries.loc[i, 'SalaryType'] == 'Yearly':
        Indian_salaries.loc[i, 'monthly_salary'] = Indian_salaries.loc[i, 'Salary']/12
    elif Indian_salaries.loc[i, 'SalaryType'] == 'Weekly':
        Indian_salaries.loc[i, 'monthly_salary'] = Indian_salaries.loc[i, 'Salary']*4

Indian_salaries.sample(5)

In [ ]:
fig, ax = plt.subplots(1,2, sharex = True)

fig.set_figheight(5)
fig.set_figwidth(20)
sns.boxplot(Indian_salaries.monthly_salary, ax= ax[0])
sns.distplot(Indian_salaries.monthly_salary, ax = ax[1])
plt.show()

In [ ]:
Indian_salaries.monthly_salary.describe().apply(lambda x: format(x, '.0f'))

- From the above numbers and plots we can see that the median salary for a professional in data science or business analytics field in India is about 58k per month
- However the max value shows that it is 2Cr, which is a not a reasonable number 
- These unreasonable numbers are called outliers
- We will learn about outliers and how to deal with them in later modules

# Conclusion:
Now that we got a good idea of how to plot, manipulate, draw insights and extract relevant data from a huge dataset, and we still have a lot of unexplored columns in the dataset, from here on look at the survey_schema data and explore data that you find interesting

All the best!   